<a href="https://colab.research.google.com/github/Merha23/AI_Tigrinya_Translation/blob/main/Final_Project_Source_Code_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Clone the GitHub Repository in Google Colab
# Authenticate Google Colab with GitHub
# Clone with Authentication
# Generate a GitHub Personal Access Token (PAT)
# How to Use Your GitHub Token in Google Colab Securely
# cloning your repository using the token:
# Use the stored token to clone your GitHub repository in Google Colab:
# Clone the Repository Securely in Colab
# Instead of using your username/password, use the token as follows:

import os
from getpass import getpass

token = getpass('Enter your GitHub Personal Access Token: ')
os.environ["GITHUB_TOKEN"] = token

repo_url = f"https://{token}@github.com/Merha23/AI_Tigrinya_Translation.git"
!git clone {repo_url}
%cd AI_Tigrinya_Translation


Enter your GitHub Personal Access Token: ··········
Cloning into 'AI_Tigrinya_Translation'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (56/56), done.
Receiving objects: 100% (56/56), 71.27 KiB | 5.09 MiB/s, done.
remote: Total 56 (delta 29), reused 0 (delta 0), pack-reused 0 (from 0)
Resolving deltas: 100% (29/29), done.
/content/AI_Tigrinya_Translation/AI_Tigrinya_Translation


In [9]:
# Run this python code and if cloning is successful, it should display the repository contents.

!ls


 Cheat_Sheet_Python_Code.ipynb		   LICENSE		      Practices.ipynb
 Final_Project_Source_Code_Program.ipynb  'Medical Translation.csv'   README.md


In [ ]:
# Mount Google Drive in Colab
# Run this command in Colab to access your Drive:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
halefom

NameError: name 'halefom' is not defined

In [14]:
# Pull the Latest Changes from GitHub
# Before pushing your changes, you need to sync your local copy of the repository with the remote one.
# Pull the latest changes:
# Run the following command to fetch and merge the latest changes from GitHub into your local branch:

!git pull https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

From https://github.com/Merha23/AI_Tigrinya_Translation
 * branch            main       -> FETCH_HEAD
Already up to date.


In [20]:
# Push Your Changes to GitHub
# After successfully pulling the latest changes and resolving any conflicts (if needed), you can now push your local changes:

!git push https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

Everything up-to-date


In [ ]:
# To reset to the remote state (undo local changes):

!git reset --hard origin/main


In [ ]:
# To fetch the latest changes without merging them:

!git fetch


In [19]:
# Push Updates from Colab to GitHub
# After editing files, commit and push:

!git config --global user.email "halefomhailemariam19@gmail.com"
!git config --global user.name "Halefom Hailemariam"
!git add .
!git commit -m "Updated final project source code program"
!git push https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [ ]:
!git add .
!git commit -m "Final_Project_Source_Code_Program.ipynb"
!git push origin main